# BentoML PyTorch MNIST Tutorial

Link to source code: https://github.com/bentoml/BentoML/tree/main/examples/pytorch_mnist/

Install required dependencies:

In [ ]:
%pip install -r requirements.txt

## Define the model

First let's define a simple PyTorch network

In [1]:
import torch
import torch.nn as nn


class SimpleConvNet(nn.Module):
    """
    Simple Convolutional Neural Network
    """

    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 10, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(26 * 26 * 10, 50),
            nn.ReLU(),
            nn.Linear(50, 20),
            nn.ReLU(),
            nn.Linear(20, 10),
        )

    def forward(self, x):
        return self.layers(x)

    def predict(self, inp):
        """predict digit for input"""
        self.eval()
        with torch.no_grad():
            raw_output = self(inp)
            _, pred = torch.max(raw_output, 1)
            return pred

## Training and Saving the model

Then we define a simple PyTorch network and some helper functions

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import transforms
from sklearn.model_selection import KFold

import bentoml

# reproducible setup for testing
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def _dataloader_init_fn(worker_id):
    np.random.seed(seed)

In [3]:
K_FOLDS = 5
NUM_EPOCHS = 5
LOSS_FUNCTION = nn.CrossEntropyLoss()


def get_dataset():
    # Prepare MNIST dataset by concatenating Train/Test part; we split later.
    train_set = MNIST(
        os.getcwd(), download=True, transform=transforms.ToTensor(), train=True
    )
    test_set = MNIST(
        os.getcwd(), download=True, transform=transforms.ToTensor(), train=False
    )
    return train_set, test_set


def train_epoch(model, optimizer, loss_function, train_loader, epoch, device="cpu"):
    # Mark training flag
    model.train()
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_function(outputs, targets)
        loss.backward()
        optimizer.step()
        if batch_idx % 499 == 0:
            print(
                "Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch,
                    batch_idx * len(inputs),
                    len(train_loader.dataset),
                    100.0 * batch_idx / len(train_loader),
                    loss.item(),
                )
            )


def test_model(model, test_loader, device="cpu"):
    correct, total = 0, 0
    model.eval()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    return correct, total

In [4]:
# load data
train_set, test_set = get_dataset()
test_loader = torch.utils.data.DataLoader(
    test_set,
    batch_size=10,
    sampler=torch.utils.data.RandomSampler(test_set),
    worker_init_fn=_dataloader_init_fn,
)

100%|██████████| 9912422/9912422 [00:00<00:00, 13577232.06it/s]


Extracting /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw/train-images-idx3-ubyte.gz to /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 606633.92it/s]


Extracting /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 3291183.10it/s]


Extracting /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 19340638.34it/s]

Extracting /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/rishab/Documents/codes/unify/bentoml/unifys_fork/BentoML/examples/pytorch_mnist/MNIST/raw



### Cross Validation

We can do some cross validation and the results can be saved with the model as metadata


In [5]:
def cross_validate(dataset, epochs=NUM_EPOCHS, k_folds=K_FOLDS, device='cpu'):
    results = {}

    # Define the K-fold Cross Validator
    kfold = KFold(n_splits=k_folds, shuffle=True)

    print("--------------------------------")

    # K-fold Cross Validation model evaluation
    for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

        print(f"FOLD {fold}")
        print("--------------------------------")

        # Sample elements randomly from a given list of ids, no replacement.
        train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
        test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

        # Define data loaders for training and testing data in this fold
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=10,
            sampler=train_subsampler,
            worker_init_fn=_dataloader_init_fn,
        )
        test_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=10,
            sampler=test_subsampler,
            worker_init_fn=_dataloader_init_fn,
        )

        # Train this fold
        model = SimpleConvNet().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
        loss_function = nn.CrossEntropyLoss()
        for epoch in range(epochs):
            train_epoch(model, optimizer, loss_function, train_loader, epoch, device)

        # Evaluation for this fold
        correct, total = test_model(model, test_loader, device)
        print("Accuracy for fold %d: %d %%" % (fold, 100.0 * correct / total))
        print("--------------------------------")
        results[fold] = 100.0 * (correct / total)

    # Print fold results
    print(f"K-FOLD CROSS VALIDATION RESULTS FOR {K_FOLDS} FOLDS")
    print("--------------------------------")
    sum = 0.0
    for key, value in results.items():
        print(f"Fold {key}: {value} %")
        sum += value

    print(f"Average: {sum/len(results.items())} %")

    return results

In [7]:
cv_results = cross_validate(train_set, epochs=1, device=device)

--------------------------------
FOLD 0
--------------------------------
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.273127
Train Epoch: 0 [4990/60000 (10%)]	Loss: 1.362216
Train Epoch: 0 [9980/60000 (21%)]	Loss: 0.411742
Train Epoch: 0 [14970/60000 (31%)]	Loss: 1.001366
Train Epoch: 0 [19960/60000 (42%)]	Loss: 0.102839
Train Epoch: 0 [24950/60000 (52%)]	Loss: 0.510927
Train Epoch: 0 [29940/60000 (62%)]	Loss: 0.053766
Train Epoch: 0 [34930/60000 (73%)]	Loss: 0.231184
Train Epoch: 0 [39920/60000 (83%)]	Loss: 0.372940
Train Epoch: 0 [44910/60000 (94%)]	Loss: 0.128234
Accuracy for fold 0: 91 %
--------------------------------
FOLD 1
--------------------------------
Train Epoch: 0 [0/60000 (0%)]	Loss: 2.327491
Train Epoch: 0 [4990/60000 (10%)]	Loss: 1.318107
Train Epoch: 0 [9980/60000 (21%)]	Loss: 0.485991
Train Epoch: 0 [14970/60000 (31%)]	Loss: 1.216311
Train Epoch: 0 [19960/60000 (42%)]	Loss: 0.419982
Train Epoch: 0 [24950/60000 (52%)]	Loss: 0.758951
Train Epoch: 0 [29940/60000 (62%)]	Loss: 0

### training the model

In [8]:
def train(dataset, epochs=NUM_EPOCHS, device="cpu"):

    train_sampler = torch.utils.data.RandomSampler(dataset)
    train_loader = torch.utils.data.DataLoader(
        dataset,
        batch_size=10,
        sampler=train_sampler,
        worker_init_fn=_dataloader_init_fn,
    )
    model = SimpleConvNet()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    loss_function = nn.CrossEntropyLoss()
    model = model.to(device)
    for epoch in range(epochs):
        train_epoch(model, optimizer, loss_function, train_loader, epoch, device)
    return model

In [9]:
trained_model = train(train_set, device=device)

Train Epoch: 0 [0/60000 (0%)]	Loss: 2.371420
Train Epoch: 0 [4990/60000 (8%)]	Loss: 1.000866
Train Epoch: 0 [9980/60000 (17%)]	Loss: 1.110684
Train Epoch: 0 [14970/60000 (25%)]	Loss: 0.737674
Train Epoch: 0 [19960/60000 (33%)]	Loss: 0.364441
Train Epoch: 0 [24950/60000 (42%)]	Loss: 0.390488
Train Epoch: 0 [29940/60000 (50%)]	Loss: 0.282015
Train Epoch: 0 [34930/60000 (58%)]	Loss: 0.199639
Train Epoch: 0 [39920/60000 (67%)]	Loss: 0.367602
Train Epoch: 0 [44910/60000 (75%)]	Loss: 0.554657
Train Epoch: 0 [49900/60000 (83%)]	Loss: 0.017670
Train Epoch: 0 [54890/60000 (91%)]	Loss: 0.086635
Train Epoch: 0 [59880/60000 (100%)]	Loss: 0.249397
Train Epoch: 1 [0/60000 (0%)]	Loss: 0.222882
Train Epoch: 1 [4990/60000 (8%)]	Loss: 0.287983
Train Epoch: 1 [9980/60000 (17%)]	Loss: 0.151059
Train Epoch: 1 [14970/60000 (25%)]	Loss: 0.118343
Train Epoch: 1 [19960/60000 (33%)]	Loss: 0.756619
Train Epoch: 1 [24950/60000 (42%)]	Loss: 0.100837
Train Epoch: 1 [29940/60000 (50%)]	Loss: 0.133997
Train Epoch: 1 

### saving the model with some metadata

In [11]:
correct, total = test_model(trained_model, test_loader, device=device)
metadata = {
    "accuracy": float(correct) / total,
    "cv_stats": cv_results,
}

tag = bentoml.pytorch.save_model(
    "pytorch_mnist",
    trained_model,
    metadata=metadata,
)

# Loading model 

In [1]:
import torch
import torch
import PIL.Image
import numpy as np
import jax
import random

import bentoml

# reproducible setup for testing
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

jax.config.update("jax_enable_x64", True)

In [2]:
img = PIL.Image.open("samples/0.png")
np_img = np.array(img)
tensor_img = torch.from_numpy(np_img).float()
tensor_img = tensor_img.unsqueeze(0).unsqueeze(0)
tensor_img = torch.nn.functional.interpolate(tensor_img, size=28, mode='bicubic', align_corners=False).to(device)
np_image = tensor_img.detach().cpu().numpy()

### load normal pytorch version

In [3]:
model = bentoml.pytorch.load_model("pytorch_mnist:latest", "cuda:0")

### load flax version
after using ivy transpile magic

In [4]:
jax_graph = bentoml.pytorch.load_model(
        "pytorch_mnist:latest", 
        "cuda:0", 
        ivy_transpile=True, 
        inputs=tensor_img
        )

2023-06-03 16:26:13.279574: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/rishab/Documents/codes/unify/ivy/ivy/func_wrapper.py:234: UserWarning: Creating many views will lead to overhead when performing inplace updates with this backend
  warnings.warn(


### inference latency comparision

In [5]:
model.to("cuda")

def _f(args):
  return model(args)

comp_model = torch.compile(_f)
_ = comp_model(tensor_img)

In [6]:
import haiku as hk

def _forward(args):
  module = jax_graph()
  return module(args)

_forward = jax.jit(_forward)
rng_key = jax.random.PRNGKey(seed)
jax_mlp_forward = hk.transform(_forward)
params = jax_mlp_forward.init(rng=rng_key, args=np_image)

In [7]:
%%timeit
_ = comp_model(tensor_img)

200 µs ± 1.67 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [8]:
%%timeit
# local jax gpu setup isn't working so these numbers are 
# jax running on cpu vs gpu
# still better latency, pretty impressive
_ = jax_mlp_forward.apply(params, None, np_image)

134 µs ± 10.8 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### sanity check if results are same

In [9]:
out_torch = comp_model(tensor_img)
out_jax = jax_mlp_forward.apply(params, None, np_image)

In [10]:
np.allclose(out_torch.detach().cpu().numpy(), out_jax, atol=1e-4)

True

## Create a BentoML Service for serving the model

Note: using `%%writefile` here because `bentoml.Service` instance must be created in a separate `.py` file

Even though we have only one model, we can create as many api endpoints as we want. Here we create two end points `predict_ndarray` and `predict_image`

In [ ]:
%%writefile service.py

import typing as t

import numpy as np
import PIL.Image
from PIL.Image import Image as PILImage

import bentoml
from bentoml.io import Image
from bentoml.io import NumpyNdarray


mnist_runner = bentoml.pytorch.get(
    "pytorch_mnist",
    name="mnist_runner",
    predict_fn_name="predict",
).to_runner()

svc = bentoml.Service(
    name="pytorch_mnist_demo",
    runners=[
        mnist_runner,
    ],
)


@svc.api(
    input=NumpyNdarray(dtype="float32", enforce_dtype=True),
    output=NumpyNdarray(dtype="int64"),
)
async def predict_ndarray(
    inp: "np.ndarray[t.Any, np.dtype[t.Any]]",
) -> "np.ndarray[t.Any, np.dtype[t.Any]]":
    assert inp.shape == (28, 28)
    # We are using greyscale image and our PyTorch model expect one
    # extra channel dimension
    inp = np.expand_dims(inp, 0)
    output_tensor = await mnist_runner.async_run(inp)
    return output_tensor.numpy()


@svc.api(input=Image(), output=NumpyNdarray(dtype="int64"))
async def predict_image(f: PILImage) -> "np.ndarray[t.Any, np.dtype[t.Any]]":
    assert isinstance(f, PILImage)
    arr = np.array(f)/255.0
    assert arr.shape == (28, 28)

    # We are using greyscale image and our PyTorch model expect one
    # extra channel dimension
    arr = np.expand_dims(arr, 0).astype("float32")
    output_tensor = await mnist_runner.async_run(arr)
    return output_tensor.numpy()


Start a dev model server to test out the service defined above

In [ ]:
!bentoml serve service.py:svc

Now you can use something like:

`curl -H "Content-Type: multipart/form-data" -F'fileobj=@samples/1.png;type=image/png' http://127.0.0.1:3000/predict_image`
    
to send an image to the digit recognition service

## Build a Bento for distribution and deployment

In [ ]:
bentoml.build(
    "service.py:svc",
    include=["*.py"],
    exclude=["tests/"],
    description="file:./README.md",
    python=dict(
        packages=["scikit-learn", "torch", "Pillow"],
    ),
)

Starting a dev server with the Bento build:

In [ ]:
!bentoml serve pytorch_mnist_demo:latest